### Train a Q&A model for Italian Language: with squad_it
* It is based on the xxl versione of the model: dbmdz/bert-base-italian-xxl-cased
* Custom training loop (to compute EM and F1 score for each epoch

The code is based on :
* the book **Natural Language Processing with Transformers** (O' Reilly, cap. 7)
* online chapter: https://huggingface.co/course/chapter7/7?fw=pt

Author: **L. Saetta**
* 31/05/2022
* v2.2

Trained on a **V100 GPU**
* takes around 1 hour

In [1]:
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline

from tqdm.auto import tqdm

# per calcolare le metriche usate in fase: evaluation
import numpy as np
import collections

# per il custom training loop
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import default_data_collator
from accelerate import Accelerator
from transformers import get_scheduler

from accelerate import Accelerator

### Load the raw train/eval datasets

In [2]:
# il dataset squad_it (in italiano), ottenuto mediante traduzione semi-automatica da SQuAd, è presente in HF Hub
raw_datasets = load_dataset("squad_it")

# carichiamo le metriche (EM, f1) usate comunemente per Squad
metric = load_metric("squad")

Using custom data configuration default
Reusing dataset squad_it (/home/datascience/.cache/huggingface/datasets/squad_it/default/0.1.0/d442bdb4794b4bae227ab19105b76d706ed7cf2ac342e4c9da4a5c36bde19d71)


In [3]:
# vediamo le "features"
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 54159
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 7609
    })
})

### Train, evaluation data preprocessing

In [4]:
# qui definisce il modello pre-trained su italian language da cui si parte
# MODEL_CHECKPOINT = "dbmdz/bert-base-italian-cased"
# switched to xxl, +1%
MODEL_CHECKPOINT = "dbmdz/bert-base-italian-xxl-cased"

MAX_LENGTH = 384
STRIDE = 128

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [6]:
#
# funzioni per il preprocessing dei dataset (train, eval)
#

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [7]:
#
# calcolo EM e F1 in fase di evaluation
#

def compute_metrics(start_logits, end_logits, features, examples):
    # questi parm. governano solo il calcolo di EM ed f1
    n_best = 20
    max_answer_length = 30
    
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [8]:
#
# valuta il modello su eval dataset
#
def evaluate_model(trainer, val_dataset, raw_val_dataset):
    # calcola i logits in inference
    predictions, _, _ = trainer.predict(val_dataset)
    start_logits, end_logits = predictions

    eval_metrics = compute_metrics(start_logits, end_logits, val_dataset, raw_val_dataset)

    print()
    print("Prestazioni su Evaluation set:")
    print(f"EM: {round(eval_metrics['exact_match'], 2)}, F1-score: {round(eval_metrics['f1'], 2)}")

### prepare train dataset

In [9]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

print(f'numero rec. in raw dataset {len(raw_datasets["train"])}, in final train set: {len(train_dataset)}')


numero rec. in raw dataset 54159, in final train set: 55137


### prepare evaluation dataset

In [10]:
validation_dataset = raw_datasets["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["test"].column_names,
)

print(f'numero rec. in raw dataset {len(raw_datasets["test"])}, in final validation set: {len(validation_dataset)}')


numero rec. in raw dataset 7609, in final validation set: 7853


### Custom training loop

In [15]:
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
EPOCHS = 2
# directory in cui si salva il modello ad ogni epoch
OUTPUT_DIR = "best_model4"

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=BATCH_SIZE,
)
eval_dataloader = DataLoader(
    validation_set, 
    collate_fn=default_data_collator, 
    batch_size=BATCH_SIZE
)

In [16]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized 

In [17]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# uso mixed mode (fp16)
accelerator = Accelerator(fp16=True)

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

num_train_epochs = EPOCHS
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [19]:
#
# qui il custom training loop 
# customer per la particolare evaluation (EM, f1) richiesta da Q&A
#

progress_bar = tqdm(range(num_training_steps))

for epoch in range(EPOCHS):
    #
    # Training
    #
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # usa tqdm per la progress bar
        progress_bar.update(1)

    #
    # Evaluation
    #
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["test"]
    )
    print(f"epoch {epoch}:", metrics)

    # Salva ad ogni epoch
    accelerator.wait_for_everyone()
    
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(OUTPUT_DIR, save_function=accelerator.save)
    
    if accelerator.is_main_process:
        tokenizer.save_pretrained(OUTPUT_DIR)


Evaluation!




epoch 0: {'exact_match': 62.715205677487184, 'f1': 74.16307519995185}
Evaluation!



epoch 1: {'exact_match': 63.56945722171113, 'f1': 75.15660367031701}


In [20]:
raw_datasets['train'][0]

{'id': '56cdca7862d2951400fa6826',
 'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
 'question': 'In quale anno si è verificato il terremoto nel Sichuan?',
 'answers': {'text': ['2008'], 'answer_start': [29]}}